In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


## Statistics and distance based features
- Calculating feature statistics by feature grup one another
- Feature derived from neighbourhood analysis

In [ ]:
rows = [[4,6,165.977125, 'Bottom_right'],[4,6,34.5395640, 'Bottom_right'],[4,6,29.1963786, 'Bottom_left'],[4,6,79.4373729, 'Bottom_left'],[4,6,290.534595, 'Bottom_right'],
       [4,6,314.412660, 'Bottom_right'],[4,6,138.9007369, 'Bottom_right'],[4,6,107.4711914, 'Bottom_right'],[4,6,242.1089786, 'Bottom_left'],[4,7,27.16719836, 'Bottom_left'],
       [4,7,413.5421978, 'Bottom_right']]

data = pd.DataFrame(rows, columns = ['User_id','Page_id','Ad_price','Ad_position'])
data

Label encode the ad_position and fit some classifier and got good score, also it could take into account all the hidden relation between variable. But no matter how good it is, it still treats all data points independently. This is why feature engineering comes into play.

In [ ]:
# Useful features
rows = [[4,6,165.977125, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],[4,6,34.5395640, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],
        [4,6,29.1963786, 'Bottom_left',476.63772, 73.711548,'Bottom_left'],[4,6,79.4373729, 'Bottom_left',476.63772, 73.711548,'Bottom_left'],
        [4,6,290.534595, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],[4,6,314.412660, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],
        [4,6,138.9007369, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],[4,6,107.4711914, 'Bottom_right',476.63772, 73.711548,'Bottom_left'],
        [4,6,242.1089786, 'Bottom_left',476.63772, 73.711548,'Bottom_left'],[4,7,27.16719836, 'Bottom_left',121.54219, 35.465202,'Bottom_left'],
       [4,7,413.5421978, 'Bottom_right',121.54219, 35.465202,'Bottom_left']]

data = pd.DataFrame(rows, columns = ['User_id','Page_id','Ad_price','Ad_position', 'Max_price','min_price','Min_price_position'])
data

It is easy to generate feature for overcome such problem. Above table it shows the useful features !

```python
    # Implementation
    gb = df.groupby(['user_id','page_id'],as_index=False).agg({'ad_price':{'max_price':np.max,'min_price':np.min}})
    gb.columns = ['user_id','page_id','min_price','max_price']
    df = pd.merge(df,gb,how='left',on=['user_id','page_id'])```
    
    
    
### More features
The main ide is to give new information

- How many pages user visited
- Standard deviation of prices
- Most visited page
- Many,many more


### Neighbors
Finding nearest neighbors
- Explicit group is not needed
- More flexible 
-  Much harder to implement


1. Number of houses in 500m,1000m,...
2. Average price per square meter in 500m, 1000m,...
3. Number of school/supermarkets/parking lots in 500m, 1000m,...
4. Distance to closest subway station

### KNN features in Springleaf
- Calculate various features from those 2000 neighbors
- Mean target of nearest 5,10,15,500,2000 neighbors
- Mean distance to 10 closest neighbors
- Mean distance to 10 closest neighbors with target 1
- Mean distance to 10 closest neighbors with target 0

## Matrix Factorizations --> for Feature Extraction
Use for text data sets !
<t> This is very close to the linear model (NMF, PCA)

## Feature Interactions

In [ ]:
rows = [['auto_part','game_news',0],['music_tickets','music_news',1],['mobile_phones','auto_blog',0]]
df = pd.DataFrame(rows, columns = ['category_ad','category_site','is_clicked'])
df

Combine the categorical feature

In [ ]:
rows = [['auto_part|game_news',0],['music_tickets|music_news',1],['mobile_phones|auto_blog',0]]
df2 = pd.DataFrame(rows, columns=['ad_site','is_clicked'])
df2

f1, f2 --> join it to become f_join, then one hot encoding it !
<t> Similar ideas can be applied to numeric feature --> multiply f1,f2, then we can get the interaction
    
### Frequent operations for feature interaction
- Multiplication
- Sum
- Diff
- Division

This will make easier for model to learn, but prone to overfit


### Practical notes
- we have a lot of possible interactions - NxN for N features.
    - Even more if use several types in interactions
    
- Need to reduce its number
    - Dimensionality Reduction
    - Feature Selection

Construct all pairwise feature interactions (sums,diffs,dots,divisions)
1. Fit random forest over them
2. Get feature importances
3. Select a few most important

Added it to original dataset !

### Extract features from DT !
HOW??

<t> The index of object leaf can be use as value for new categoric feature
    
#### How to use it ?
```python
# In sklearn    
tree_model.apply()
    
# In xgboost
booster.predict(pred_leaf=True)```

## t-SNE
Visualing data for generated features --> THIS IS GREAT TOOL FOR VISUALIZATION
<T> Non-linear model for dimensionality reduction

Manifold learning methods !! --> Tries to project from high dimensional space into small dimensional place

#### Interpretation of tSNE ??
tSNE results strongly depends on its parameter (e.g. Perplexity)

#### How to Use t-SNE effectively --> can obtain new feature from t-SNE plot
Martin Wattenberg, https://distill.pub/2016/misread-tsne/

### Practical note
1. Result heavily depends on hpyerparameters (perplexity)
    - Good practice is to use several projections with different perplexities (5-100)
    
2. Due to stochastic nature, tSNE provides different projections even for the same data\hyperparameters
    - Train and test should be projected together
    
3. tSNE runs for a long time with a big number of features
    - it is common to do dimensionality reduction before projection